<a href="https://colab.research.google.com/github/nanpolend/365E5/blob/master/%E5%B0%8B%E6%89%BE%E9%A1%9E%E5%9C%B0%E6%98%9F%E7%90%83.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import json
import numpy as np
from PIL import Image
from io import BytesIO

# 获取NASA API密钥
nasa_api_key = 'YOUR_API_KEY'

# 搜索类地行星图片的API
search_url = 'https://images-api.nasa.gov/search?q=earth&media_type=image'

# 发送请求并获取JSON数据
response = requests.get(search_url)
data = json.loads(response.text)

# 初始化变量
earth_img = None
earth_features = None
similarity_scores = {}

# 循环所有图像
for item in data['collection']['items']:
    # 获取图像信息和链接
    img_info_url = item['href']
    img_info_response = requests.get(img_info_url)
    img_info_data = json.loads(img_info_response.text)
    img_url = img_info_data[0]['links'][0]['href']

    # 下载图片并将其转换为numpy数组
    img_response = requests.get(img_url)
    img = Image.open(BytesIO(img_response.content))
    img_array = np.array(img)

    # 如果是地球图像，将其保存为参考图像并计算其特征
    if item['data'][0]['title'] == 'Earth':
        earth_img = img
        earth_features = np.mean(img_array, axis=(0,1))
    else:
        # 计算图像与地球的相似度得分
        similarity_score = np.sum(np.abs(np.mean(img_array, axis=(0,1)) - earth_features))
        similarity_scores[item['data'][0]['title']] = similarity_score

# 打印前十个最相似的行星
top_matches = sorted(similarity_scores.items(), key=lambda x: x[1])[:10]
for match in top_matches:
    print(match[0], match[1])


TypeError: ignored